In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')

youtube_frames_path = "/content/drive/MyDrive/VisionAssist-Dataset/Yt Frames /Example_4_frames"

if os.path.exists(youtube_frames_path):
    print(f"✅ Google Drive mounted and YouTube frames folder found at: {youtube_frames_path}")
else:
    print(f"⚠️ ERROR: Could not find YouTube frames folder at: {youtube_frames_path}")
    print("Please double-check the path and try mounting again.")

Mounted at /content/drive
✅ Google Drive mounted and YouTube frames folder found at: /content/drive/MyDrive/VisionAssist-Dataset/Yt Frames /Example_4_frames


In [ ]:
import zipfile



# --- Define Paths ---
coco_zip_path = "/content/drive/MyDrive/VisionAssist-Dataset/filtered_coco_data.zip"
unzip_destination = "temp_coco_unzipped" # Temporary folder in Colab

# --- Check if Zip Exists ---
if not os.path.exists(coco_zip_path):
    print(f"⚠️ ERROR: Cannot find the zip file at: {coco_zip_path}")
    print("Please make sure the file exists in your Google Drive.")
else:
    # --- Unzip the File ---
    print(f"Unzipping '{os.path.basename(coco_zip_path)}' to '{unzip_destination}'...")
    # Create the destination folder if it doesn't exist
    os.makedirs(unzip_destination, exist_ok=True)
    try:
        with zipfile.ZipFile(coco_zip_path, 'r') as zip_ref:
            zip_ref.extractall(unzip_destination)
        print("✅ Unzipping complete.")

        potential_coco_data_folder = os.path.join(unzip_destination, "yolo_data")
        if os.path.exists(os.path.join(potential_coco_data_folder, "images")):
             actual_coco_data_folder = potential_coco_data_folder
             print(f"Found COCO data inside '{actual_coco_data_folder}'")
        elif os.path.exists(os.path.join(unzip_destination, "images")):
             actual_coco_data_folder = unzip_destination
             print(f"Found COCO data directly inside '{actual_coco_data_folder}'")
        else:
             actual_coco_data_folder = None
             print(f"⚠️ ERROR: Could not find 'images' and 'labels' folders within '{unzip_destination}'.")

    except zipfile.BadZipFile:
        print(f"⚠️ ERROR: The file '{coco_zip_path}' is not a valid zip file or is corrupted.")
        actual_coco_data_folder = None
    except Exception as e:
        print(f"⚠️ An error occurred during unzipping: {e}")
        actual_coco_data_folder = None

Unzipping 'filtered_coco_data.zip' to 'temp_coco_unzipped'...
✅ Unzipping complete.
Found COCO data directly inside 'temp_coco_unzipped'


In [ ]:
import shutil
import os

# --- Define Paths ---
if 'actual_coco_data_folder' not in locals() or actual_coco_data_folder is None:
    print("⚠️ ERROR: 'actual_coco_data_folder' is not defined. Please run Cell 1 successfully first.")
else:
    coco_data_folder = actual_coco_data_folder # Use the path found after unzipping
    youtube_frames_folder = "/content/drive/MyDrive/VisionAssist-Dataset/Yt Frames /Example_4_frames" # Adjust if needed
    master_dataset_folder = "master_dataset"
    master_images_folder = os.path.join(master_dataset_folder, "images")
    # master_labels_folder = os.path.join(master_dataset_folder, "labels") # We are not creating labels folder yet

    # --- Create Master Folders ---
    print(f"\nCreating master images folder at '{master_images_folder}'...")
    os.makedirs(master_images_folder, exist_ok=True)

    coco_images_path = os.path.join(coco_data_folder, "images")
    # coco_labels_path = os.path.join(coco_data_folder, "labels") # Path not needed

    # ---  Only check for images path ---
    if os.path.exists(coco_images_path):
        print(f"Copying images from '{coco_images_path}' to '{master_images_folder}'...")
        # Using copytree with dirs_exist_ok=True merges contents
        shutil.copytree(coco_images_path, master_images_folder, dirs_exist_ok=True)
        print("COCO images copied.")

    else:
        # --- Error message adjusted ---
        print(f"⚠️ ERROR: Could not find COCO 'images' folder in '{coco_data_folder}'.")

    # --- Copy YouTube Frames ---
    if os.path.exists(youtube_frames_folder):
        print(f"Copying images from '{youtube_frames_folder}' to '{master_images_folder}'...")
        copied_yt_count = 0
        for filename in os.listdir(youtube_frames_folder):
            source_file = os.path.join(youtube_frames_folder, filename)
            if os.path.isfile(source_file) and filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                destination_file = os.path.join(master_images_folder, filename)
                shutil.copy2(source_file, destination_file)
                copied_yt_count += 1
        print(f"Copied {copied_yt_count} YouTube frame images.")
    else:
         print(f"⚠️ Skipping YouTube frames copy as folder '{youtube_frames_folder}' was not found.")


    # --- Verification ---
    print("\n--- Verification ---")
    try:
        num_master_images = len(os.listdir(master_images_folder))
        print(f"Total images in '{master_images_folder}': {num_master_images}")
        print("\n✅ Master dataset folder with images created successfully.")
        print(f"You can now point your YOLOv8 annotation script to the '{master_images_folder}'.")
        print(f"Make sure your annotation script saves the *new* label files into a separate 'labels' folder (e.g., 'master_dataset/labels').")
    except Exception as e:
        print(f"⚠️ Verification failed: {e}")


Creating master images folder at 'master_dataset/images'...
Copying images from 'temp_coco_unzipped/images' to 'master_dataset/images'...
COCO images copied.
Copying images from '/content/drive/MyDrive/VisionAssist-Dataset/Yt Frames /Example_4_frames' to 'master_dataset/images'...
Copied 2138 YouTube frame images.

--- Verification ---
Total images in 'master_dataset/images': 7138

✅ Master dataset folder with images created successfully.
You can now point your YOLOv8 annotation script to the 'master_dataset/images'.
Make sure your annotation script saves the *new* label files into a separate 'labels' folder (e.g., 'master_dataset/labels').


In [ ]:
# Cell for Annotation

# --- 1. Install Ultralytics ---
!pip install ultralytics -q
import torch
from ultralytics import YOLO
import os
from tqdm import tqdm # Progress bar

print("✅ Ultralytics installed.")

# --- 2. Define Paths ---
MASTER_IMAGES_FOLDER = "/content/master_dataset/images"
MASTER_LABELS_FOLDER = "/content/master_dataset/labels" # Labels will be created/overwritten here

# --- 3. Create Labels Folder ---
os.makedirs(MASTER_LABELS_FOLDER, exist_ok=True)
print(f"Labels will be saved to: {MASTER_LABELS_FOLDER}")

# --- 4. Load YOLOv8 Model for Annotation ---
ANNOTATION_MODEL_NAME = 'yolov8n.pt'
print(f"Loading annotation model: {ANNOTATION_MODEL_NAME}...")
annotation_model = YOLO(ANNOTATION_MODEL_NAME)
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
annotation_model.to(DEVICE)
print(f"Model loaded successfully on {DEVICE}.")

# --- 5. Class Mapping (REMOVED - Not needed for this approach) ---
print("Will save labels for ALL detected COCO classes (0-79) using their original IDs.")

# --- 6. Get List of Images ---
try:
    # Gets all image files from the master images folder
    image_files = [f for f in os.listdir(MASTER_IMAGES_FOLDER) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    print(f"\nFound {len(image_files)} total images to annotate in {MASTER_IMAGES_FOLDER}")
except FileNotFoundError:
    print(f"--- ERROR ---")
    print(f"Could not find the image folder: {MASTER_IMAGES_FOLDER}")
    print("Please ensure the master_dataset folder was created correctly.")
    image_files = [] # Prevent crash

# --- 7. Annotation Loop (Processes ALL Images) ---
annotation_count = 0 # Counter for images where labels were successfully generated

if image_files: # Only proceed if images were found
    # Loop through every image file with a progress bar
    for image_name in tqdm(image_files, desc="Annotating ALL Images"):
        image_path = os.path.join(MASTER_IMAGES_FOLDER, image_name)

        # Define the path where the label file should be saved
        label_name = os.path.splitext(image_name)[0] + ".txt"
        label_path = os.path.join(MASTER_LABELS_FOLDER, label_name)




        # Predicts objects in the current image using the loaded model
        results = annotation_model.predict(image_path, save=False, verbose=False, conf=0.25)

        # --- Convert Results to YOLO TXT Format and Save ---
        # Checks if the prediction found any objects
        if results and len(results[0].boxes) > 0:
            lines_written = 0
            # Opens the corresponding label file for writing (overwrites if it exists)
            with open(label_path, 'w') as f:
                # Iterates through each detected object (box)
                for box in results[0].boxes:
                    # Gets the original COCO class ID (0-79)
                    coco_class_id = int(box.cls)

                    # Gets normalized coordinates [center_x, center_y, width, height]
                    x_center, y_center, width, height = box.xywhn[0].tolist()

                    # Writes the label line using the original COCO ID
                    f.write(f"{coco_class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")
                    lines_written += 1 # Count line written

            # If we wrote labels to the file, increment the success counter
            if lines_written > 0:
                 annotation_count += 1
            # If no objects were detected (lines_written == 0),
            # but an empty file was created, remove the empty file.
            elif os.path.exists(label_path):
                 try: # Add error handling just in case
                     if os.path.getsize(label_path) == 0:
                          os.remove(label_path)
                 except OSError:
                      pass # Ignore error if file is already gone etc.

        # else: # No detections made by the model for this image
            # No label file needed or created.

    # --- Final Summary ---
    print(f"\n--- Annotation Summary ---")
    print(f"Processed {len(image_files)} images.")
    print(f"Generated or overwrote labels for {annotation_count} images.")
    print(f"Labels were saved using original COCO class IDs (0-79).")
    print(f"All labels are now saved in: {MASTER_LABELS_FOLDER}")
else:
    # Message if the image folder was empty or not found
    print("No images found to process.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 73.6 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
✅ Ultralytics installed.
Labels will be saved to: /content/master_dataset/labels
Loading annotation model: yolov8n.pt...
Model loaded successfully on cuda.
Will save labels for ALL detected COCO classes (0-79) using their original IDs.

Found 7138 total images to annotate in /content/master_dataset/images


Annotating ALL Images: 100%|██████████| 7138/7138 [02:11<00:00, 54.15it/s]


--- Annotation Summary ---
Processed 7138 images.
Generated or overwrote labels for 7102 images.
Labels were saved using original COCO class IDs (0-79).
All labels are now saved in: /content/master_dataset/labels


Splitting the dataset

In [ ]:
import os
import random
import shutil
from tqdm import tqdm

# --- Configuration ---
MASTER_DATASET_FOLDER = "master_dataset" # Folder containing 'images' and 'labels'
OUTPUT_FOLDER = "split_dataset"        # New folder where 'train', 'valid', 'test' will be created
TRAIN_RATIO = 0.70                     # 70% for training
VALID_RATIO = 0.20                     # 20% for validation
TEST_RATIO = 0.10                      # 10% for testing

# --- Sanity Check ---
if abs(TRAIN_RATIO + VALID_RATIO + TEST_RATIO - 1.0) > 1e-6:
    print("⚠️ ERROR: Ratios must add up to 1.0")
else:
    # --- Define Input and Output Paths ---
    images_input_dir = os.path.join(MASTER_DATASET_FOLDER, "images")
    labels_input_dir = os.path.join(MASTER_DATASET_FOLDER, "labels")

    train_images_dir = os.path.join(OUTPUT_FOLDER, "train", "images")
    train_labels_dir = os.path.join(OUTPUT_FOLDER, "train", "labels")
    valid_images_dir = os.path.join(OUTPUT_FOLDER, "valid", "images")
    valid_labels_dir = os.path.join(OUTPUT_FOLDER, "valid", "labels")
    test_images_dir = os.path.join(OUTPUT_FOLDER, "test", "images")
    test_labels_dir = os.path.join(OUTPUT_FOLDER, "test", "labels")

    # --- Create Output Directories ---
    print(f"Creating output directories in '{OUTPUT_FOLDER}'...")
    os.makedirs(train_images_dir, exist_ok=True)
    os.makedirs(train_labels_dir, exist_ok=True)
    os.makedirs(valid_images_dir, exist_ok=True)
    os.makedirs(valid_labels_dir, exist_ok=True)
    os.makedirs(test_images_dir, exist_ok=True)
    os.makedirs(test_labels_dir, exist_ok=True)

    # --- Get List of All Image Files ---
    try:
        all_images = [f for f in os.listdir(images_input_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        if not all_images:
             print(f"⚠️ ERROR: No images found in '{images_input_dir}'.")
        else:
             print(f"Found {len(all_images)} images in '{images_input_dir}'.")
             # --- Shuffle the Images ---
             random.shuffle(all_images)

             # --- Calculate Split Indices ---
             total_images = len(all_images)
             train_end_idx = int(total_images * TRAIN_RATIO)
             valid_end_idx = train_end_idx + int(total_images * VALID_RATIO)
             # The rest go to test

             # --- Define Splits ---
             train_files = all_images[:train_end_idx]
             valid_files = all_images[train_end_idx:valid_end_idx]
             test_files = all_images[valid_end_idx:]

             print(f"Splitting into: {len(train_files)} Train, {len(valid_files)} Valid, {len(test_files)} Test")

             # --- Function to Copy Files ---
             def copy_files(file_list, source_img_dir, source_lbl_dir, dest_img_dir, dest_lbl_dir):
                 copied_count = 0
                 for filename in tqdm(file_list, desc=f"Copying to {os.path.basename(dest_img_dir)}"):
                     # Copy Image
                     source_image_path = os.path.join(source_img_dir, filename)
                     dest_image_path = os.path.join(dest_img_dir, filename)
                     if os.path.exists(source_image_path):
                         shutil.copy2(source_image_path, dest_image_path)

                         # Copy Corresponding Label (if it exists)
                         label_filename = os.path.splitext(filename)[0] + ".txt"
                         source_label_path = os.path.join(source_lbl_dir, label_filename)
                         dest_label_path = os.path.join(dest_lbl_dir, label_filename)
                         if os.path.exists(source_label_path):
                             shutil.copy2(source_label_path, dest_label_path)
                         # else:
                         #     print(f"Warning: Label file not found for {filename}") # Optional warning
                         copied_count += 1
                 return copied_count

             # --- Perform the Copying ---
             print("\nCopying training files...")
             train_copied = copy_files(train_files, images_input_dir, labels_input_dir, train_images_dir, train_labels_dir)
             print(f"Copied {train_copied} training images and labels.")

             print("\nCopying validation files...")
             valid_copied = copy_files(valid_files, images_input_dir, labels_input_dir, valid_images_dir, valid_labels_dir)
             print(f"Copied {valid_copied} validation images and labels.")

             print("\nCopying test files...")
             test_copied = copy_files(test_files, images_input_dir, labels_input_dir, test_images_dir, test_labels_dir)
             print(f"Copied {test_copied} test images and labels.")

             print("\n✅ Dataset splitting complete!")
             print(f"Split dataset is ready in the '{OUTPUT_FOLDER}' folder.")

    except FileNotFoundError:
        print(f"--- ERROR ---")
        print(f"Could not find the image folder: {images_input_dir}")
        print("Please ensure the master_dataset folder exists and contains images.")
    except Exception as e:
        print(f"--- An unexpected error occurred: {e} ---")

Creating output directories in 'split_dataset'...
Found 7138 images in 'master_dataset/images'.
Splitting into: 4996 Train, 1427 Valid, 715 Test

Copying training files...


Copying to images: 100%|██████████| 4996/4996 [00:01<00:00, 3575.53it/s]


Copied 4996 training images and labels.

Copying validation files...


Copying to images: 100%|██████████| 1427/1427 [00:00<00:00, 3321.24it/s]


Copied 1427 validation images and labels.

Copying test files...


Copying to images: 100%|██████████| 715/715 [00:00<00:00, 2890.17it/s]

Copied 715 test images and labels.

✅ Dataset splitting complete!
Split dataset is ready in the 'split_dataset' folder.


Creating a data.yaml file


In [ ]:
%%writefile coco_custom_data.yaml
# YAML configuration file for YOLOv8 training

# Dataset paths: Relative paths to the split dataset folder
# Assumes this yaml file is in /content/ and the data is in /content/split_dataset/
train: ./split_dataset/train/images
val: ./split_dataset/valid/images
test: ./split_dataset/test/images # Optional: path to test images

# Number of classes
# Must be 80 since we used original COCO IDs during annotation
nc: 80

# Class names
# Must be the standard 80 COCO class names in the correct order (0-79)
names:
  0: person
  1: bicycle
  2: car
  3: motorcycle
  4: airplane
  5: bus
  6: train
  7: truck
  8: boat
  9: traffic light
  10: fire hydrant
  11: stop sign
  12: parking meter
  13: bench
  14: bird
  15: cat
  16: dog
  17: horse
  18: sheep
  19: cow
  20: elephant
  21: bear
  22: zebra
  23: giraffe
  24: backpack
  25: umbrella
  26: handbag
  27: tie
  28: suitcase
  29: frisbee
  30: skis
  31: snowboard
  32: sports ball
  33: kite
  34: baseball bat
  35: baseball glove
  36: skateboard
  37: surfboard
  38: tennis racket
  39: bottle
  40: wine glass
  41: cup
  42: fork
  43: knife
  44: spoon
  45: bowl
  46: banana
  47: apple
  48: sandwich
  49: orange
  50: broccoli
  51: carrot
  52: hot dog
  53: pizza
  54: donut
  55: cake
  56: chair
  57: couch
  58: potted plant
  59: bed
  60: dining table
  61: toilet
  62: tv
  63: laptop
  64: mouse
  65: remote
  66: keyboard
  67: cell phone
  68: microwave
  69: oven
  70: toaster
  71: sink
  72: refrigerator
  73: book
  74: clock
  75: vase
  76: scissors
  77: teddy bear
  78: hair drier
  79: toothbrush

Writing coco_custom_data.yaml


### Fine Tuning

In [ ]:
# --- Start YOLOv8 Training ---

!yolo task=detect mode=train \
model=yolov8n.pt \
data=coco_custom_data.yaml \
epochs=50 \
imgsz=640 \
name=yolov8n_custom_coco_run

Ultralytics 8.3.222 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=coco_custom_data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8n_custom_coco_run, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=T

In [ ]:
# Cell to Save Trained Model to Google Drive

import os
import shutil
from google.colab import drive

# --- Define Paths ---
# Path where the training script saved the best model
trained_model_path_colab = '/content/runs/detect/yolov8n_custom_coco_run/weights/best.pt'

# Path where you want to save the model in Google Drive
drive_save_folder = '/content/drive/My Drive/VisionAssist-Models/' # Example folder
model_save_name = 'yolov8n_custom_coco_best.pt' # Choose a name for the saved model
drive_save_path = os.path.join(drive_save_folder, model_save_name)

# --- Mount Drive (if not already mounted) ---
try:
    if not os.path.exists('/content/drive/My Drive'):
         print("Mounting Google Drive...")
         drive.mount('/content/drive')
    else:
         print("Google Drive already mounted.")
except Exception as e:
    print(f"Error mounting drive: {e}")

# --- Check if Model Exists ---
if not os.path.exists(trained_model_path_colab):
    print(f"--- ERROR: Trained model not found at '{trained_model_path_colab}' ---")
    print("Please ensure training completed successfully and check the path.")
else:
    # --- Create Save Directory in Drive ---
    print(f"Ensuring save directory exists: {drive_save_folder}")
    os.makedirs(drive_save_folder, exist_ok=True)

    # --- Copy the Model ---
    print(f"Copying '{os.path.basename(trained_model_path_colab)}' to '{drive_save_path}'...")
    try:
        shutil.copy(trained_model_path_colab, drive_save_path)
        print("✅ Model successfully saved to Google Drive.")
    except Exception as e:
        print(f"--- ERROR saving model to Drive: {e} ---")

Google Drive already mounted.
Ensuring save directory exists: /content/drive/My Drive/VisionAssist-Models/
Copying 'best.pt' to '/content/drive/My Drive/VisionAssist-Models/yolov8n_custom_coco_best.pt'...
✅ Model successfully saved to Google Drive.
